In [2]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [3]:
def split_time(df):
    sub = df['patient_id'].split('-')
    try:
        time = int(sub[2])
        return time
    except IndexError:
        return 0

In [4]:
def remove_time(df):
    patient = df['patient_id'].split('-')
    return '{}-{}'.format(patient[0], patient[1])

In [5]:
def remove_day(x):
    if isinstance(x, str):
        x = x.replace('day ', '')
    return x

In [6]:
add_info = pd.read_csv('C:/Users/lzoeckler/Desktop/namibia_PCR/formatted_namibia_PCR.csv')
add_info.head()

,participant_id,age_yrs,sex,day0_enrollee,days_since_detection_enroll,time_to_negativity_urdt,where_detected,timepoint_days,quansys_HRP2_pg_ml,qpcr_p_density
0,pa-001,2,Female,1,0,29,Health Facility,0,NaN,NaN
1,pa-002,80,Female,1,0,35,Health Facility,0,43928722.05,NaN
2,pa-003,23,Female,0,5,66,Health Facility,0,251849.37,NaN
3,pa-004,17,Female,1,0,56,Health Facility,0,46391.5,NaN
4,pa-005,21,Male,1,0,71,Health Facility,0,7872641.8,NaN


In [8]:
add_info.drop_duplicates(subset=['participant_id', 'timepoint_days'], inplace=True, keep='last')
add_info.head()

,participant_id,age_yrs,sex,day0_enrollee,days_since_detection_enroll,time_to_negativity_urdt,where_detected,timepoint_days,quansys_HRP2_pg_ml,qpcr_p_density
0,pa-001,2,Female,1,0,29,Health Facility,0,NaN,NaN
1,pa-002,80,Female,1,0,35,Health Facility,0,43928722.05,NaN
2,pa-003,23,Female,0,5,66,Health Facility,0,251849.37,NaN
3,pa-004,17,Female,1,0,56,Health Facility,0,46391.5,NaN
4,pa-005,21,Male,1,0,71,Health Facility,0,7872641.8,NaN


In [11]:
time_df = pd.read_csv('C:/Users/lzoeckler/Desktop/all_data/separated/18-UCSF-01.csv')
time_df = time_df[['participant_id', 'timepoint_days', 'age_yrs', 'quansys_HRP2_pg_ml']]
time_df = time_df.merge(add_info, how='inner', on=['participant_id', 'timepoint_days'])
time_df.sort_values(['participant_id', 'timepoint_days'], inplace=True)
time_df.set_index(['participant_id', 'timepoint_days'], inplace=True)
time_df.head()

age_yrs_x quansys_HRP2_pg_ml_x  age_yrs_y  \
participant_id timepoint_days                                              
pa-001         3                       2            318702.59          2   
               7                       2             73737.79          2   
               14                      2              4138.79          2   
               21                      2              1377.18          2   
               28                      2              2024.41          2   

                                  sex  day0_enrollee  \
participant_id timepoint_days                          
pa-001         3               Female              1   
               7               Female              1   
               14              Female              1   
               21              Female              1   
               28              Female              1   

                               days_since_detection_enroll  \
participant_id timepoint_days                                
pa-001         3                                         0   
               7                                         0   
               14                                        0   
               21                                        0   
               28                                        0   

                               time_to_negativity_urdt   where_detected  \
participant_id timepoint_days                                             
pa-001         3                                    29  Health Facility   
               7                                    29  Health Facility   
               14                                   29  Health Facility   
               21                                   29  Health Facility   
               28                                   29  Health Facility   

                              quansys_HRP2_pg_ml_y  qpcr_p_density  
participant_id timepoint_days                                       
pa-001         3                         318702.59             NaN  
               7                          73737.79             NaN  
               14                          4138.79             NaN  
               21                          1377.18             NaN  
               28                              NaN             NaN

In [141]:
time_df.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/final_dilutions.csv')

In [142]:
partial_format = samples_data.copy(deep=True)
partial_format['time_point_days'] = partial_format.apply(split_time, axis=1)
partial_format['patient_id'] = partial_format.apply(remove_time, axis=1)
partial_format = partial_format.merge(add_info, how='left', on=['patient_id', 'time_point_days'])
partial_format.sort_values(['patient_id', 'time_point_days'], inplace=True)
partial_format.set_index(['patient_id', 'time_point_days'], inplace=True)
partial_format.head()

well error     HRP2_pg_ml    LDH_Pan_pg_ml  \
patient_id time_point_days                                              
pa-001     3                 g6   NaN         145.95           142.30   
           3                 f3   NaN      397468.45     < 1801250.00   
           3                 e3   NaN      318702.59         74360.14   
           3                 h3   NaN  1537195777.01  119019730410.15   
           3                 h6   NaN       10273.61          2535.11   

                              LDH_Pv_pg_ml        CRP_ng_ml concentration  \
patient_id time_point_days                                                  
pa-001     3                         15.35          5678.58             1   
           3                   < 291250.00     < 3342500.00        125000   
           3                      10385.14       < 66850.00          2500   
           3                16185275422.09  286153421264.94     312500000   
           3                        733.44         49491.76            50   

                            day0_enrollee  days_since_tx  initial_sample  \
patient_id time_point_days                                                 
pa-001     3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   

                            got_PQ  age     sex  fever48_r    enrolled_from  \
patient_id time_point_days                                                    
pa-001     3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   

                            returned_with_fever when_returned_with_fever  \
patient_id time_point_days                                                 
pa-001     3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            

                            retreated when_retreated  
patient_id time_point_days                            
pa-001     3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN

In [143]:
partial_format.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/partially_formatted.csv')